In [1]:
import pandas as pd
import os

In [ ]:
# source: https://educacion.gob.ec/base-de-datos/

In [2]:
carpeta_inicio = "Inicio"
nombre_columna_distintiva = "Cod_Provincia"
encontrado_provincia = False

archivos_csv = [archivo for archivo in os.listdir(carpeta_inicio) if archivo.endswith(".csv")]
provincias_unicas = pd.DataFrame(columns=["Cod_Provincia", "Provincia"])
resultado_global = pd.DataFrame()

In [3]:
for archivo in archivos_csv:
    nombre_columna = archivo.split("_")[-1].split(".csv")[0]
    ruta_archivo = os.path.join(carpeta_inicio, archivo)

    header_row = next(i for i, linea in enumerate(open(ruta_archivo, 'r', encoding='ISO-8859-1')) if nombre_columna_distintiva in linea)
    
    df = pd.read_csv(ruta_archivo, skiprows=header_row, encoding='ISO-8859-1', delimiter=';')
    if "Total Estudiantes" in df.columns:
        df.rename(columns={"Total Estudiantes": "Total_Estudiantes"}, inplace=True)

    resultado = df.groupby('Cod_Provincia')['Total_Estudiantes'].sum().astype(int).astype(str).reset_index()    
    resultado = resultado.rename(columns={'Total_Estudiantes': f'{nombre_columna}'})
    
    if "Provincia" in df.columns and not encontrado_provincia:
        provincias_unicas = df[["Cod_Provincia", "Provincia"]].drop_duplicates()
        encontrado_provincia = True
    
    #print(provincias_unicas)
    #print(type(provincias_unicas))
    
    if resultado_global.empty:
        resultado_global = resultado
    else:
        resultado_global = resultado_global.merge(resultado, on='Cod_Provincia', how='outer')

In [4]:
resultado_provincias = resultado_global.merge(provincias_unicas, on='Cod_Provincia', how='left')

#print(resultado_global)
#print(provincias_unicas)

resultado_provincias['Provincia'] = resultado_provincias['Provincia'].apply(lambda x: x[0] if isinstance(x, list) else x)
resultado_provincias = resultado_provincias[resultado_provincias['Cod_Provincia'] != 90]
resultado_provincias = resultado_provincias.reindex(sorted(resultado_provincias.columns), axis=1)
resultado_provincias = resultado_provincias[['Cod_Provincia'] + ['Provincia'] + [col for col in resultado_provincias.columns if col not in ['Cod_Provincia', 'Provincia']]]

#print(resultado_provincias)

    Cod_Provincia                       Provincia 2009-2010Inicio  \
0               1                           AZUAY          200034   
1               2                         BOLIVAR           59292   
2               3                          CAÃAR           71945   
3               4                          CARCHI           45972   
4               5                        COTOPAXI          120260   
5               6                      CHIMBORAZO          137144   
6               7                          EL ORO          172640   
7               8                      ESMERALDAS          183636   
8               9                          GUAYAS          961491   
9              10                        IMBABURA          120771   
10             11                            LOJA          133579   
11             12                        LOS RIOS          234018   
12             13                          MANABI          398805   
13             14                 

In [5]:
resultado_provincias.to_csv('resultadosF.csv', index=False)